# Continuous data generation

In [0]:
import random
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.preprocessing
import matplotlib.pyplot as plt
# plt.rc('text', usetex=True)
# plt.rc('font', **{'family': "sans-serif"})
# params = {
#     'text.latex.preamble':
#     [r'\usepackage{amsmath}', r'\usepackage{amsfonts}', r'\usepackage{bm}']
# }
# plt.rcParams.update(params)
# #from matplotlib2tikz import save as tikz_save
# #from IPython.display import set_matplotlib_formats
# #set_matplotlib_formats('pdf', 'png')
# plt.rcParams['savefig.dpi'] = 75

# plt.rcParams['figure.autolayout'] = False
# plt.rcParams['figure.figsize'] = 10, 6
# plt.rcParams['axes.labelsize'] = 18
# plt.rcParams['axes.titlesize'] = 20
# plt.rcParams['font.size'] = 16
# plt.rcParams['lines.linewidth'] = 2.0
# plt.rcParams['lines.markersize'] = 8
# plt.rcParams['legend.fontsize'] = 14

# plt.rcParams['text.usetex'] = True
# plt.rcParams['font.family'] = "serif"
# plt.rcParams['font.serif'] = "cm"

In [0]:
# n: sample size; d1: continuous features' dimension;
# x_quant: U([0,1]); xd_quant: "true" generating data of Y
n = 1000
d1 = 2
cuts = ([0,0.333,0.666,1])
replicates = 1
# The true logistic regression parameter is fixed
if d1>0: 
    theta_quant = np.array([[0]*d1]*(len(cuts)-1))
    theta_quant[1,:] = 2
    theta_quant[2,:] = -2

In [0]:
def generate_data(seed=0):
    random.seed(seed)
    x_quant = np.array(np.random.uniform(size=(n,d1)))
    xd_quant = np.ndarray.copy(x_quant)
    # x_quant is cut in thirds into xd_quant
    for i in range(d1):
        xd_quant[:,i] = pd.cut(x_quant[:,i],bins=cuts,labels=[0,1,2])
# The log odd probabilities of Y given each x_i can be exactly calculated
    log_odd = np.array([0]*n)
    for i in range(n):
        for j in range(d1):
            log_odd[i] += theta_quant[int(xd_quant[i,j]),j]
# Y is then drawn from this pdf
    p = 1/(1+np.exp(-log_odd))
    y = np.random.binomial(1,p)

    return([x_quant,y])

# Neural network architecture

In [5]:
# We rely on Keras for all calculations and the TensorFlow backend.
import tensorflow as tf
sess = tf.Session()
from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


In [0]:
from keras import *
from keras.layers import *
from keras.callbacks import LambdaCallback, Callback, ReduceLROnPlateau , TensorBoard
import sklearn.linear_model    

In [0]:
m_quant = [3,3]

def initialize_neural_net():
    liste_inputs_quant = [None]*d1
    liste_layers_quant = [None]*d1
    liste_layers_quant_inputs = [None]*d1


    for i in range(d1):
        liste_inputs_quant[i] = Input((1,))
        liste_layers_quant[i] = Dense(m_quant[i],activation ='softmax')
        liste_layers_quant_inputs[i] = liste_layers_quant[i](liste_inputs_quant[i])

    return([liste_inputs_quant,liste_layers_quant,liste_layers_quant_inputs])

In [0]:
def evaluate_disc(prob=[]):
        
    results = [None]*d1
    
    X_transformed = np.ones((n,1))

    for j in range(d1):
        results[j] = np.argmax(prob[j][0],axis=1)
        X_transformed = np.concatenate((X_transformed,sk.preprocessing.OneHotEncoder().fit_transform(X=results[j].reshape(-1, 1)).toarray()),axis=1)
        
    proposed_logistic_regression = sk.linear_model.LogisticRegression(fit_intercept=False)
    
    proposed_logistic_regression.fit(X=X_transformed,y=y.reshape((n,)))
    
    BIC_training = 2*sk.metrics.log_loss(y, proposed_logistic_regression.predict_proba(X=X_transformed)[:,1],normalize=False) + proposed_logistic_regression.coef_.shape[1]*np.log(n)
    
    return(BIC_training)

In [0]:
class LossHistory(Callback):

    def on_train_begin(self, logs={}):
        self.losses = []
        self.best_criterion = float("inf")
        self.best_outputs = []
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        outputs = []
        
        for j in range(d1):
            outputs.append(K.function([liste_layers_quant[j].input],[liste_layers_quant[j].output])([x_quant[:,j,np.newaxis]]))
#             if self.epochs in [1,2,3,4,5,10,20,30,50,100,200,250,299,300]:
#                 plt.xlim((0,1))
#                 plt.ylim((0,1))
#                 for k in range(outputs[j][0].shape[1]):
#                     plt.plot(np.sort(x_quant[:,j]),outputs[j][0][np.argsort(x_quant[:,j]),k], label = r'${q}_{\bm{\alpha}_{'+str(j)+','+str(k)+'}}$', color = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'][k+2])
#                 plt.title('Continuous feature '+str(j)+' at iteration ' + str(self.epochs))
#                 plt.ylabel(r'${q}_{\bm{\alpha}_{'+str(j)+',h}}$')
#                 plt.xlabel(r'$x_1$')
#                 plt.axvline(x=0.33333, label = r'$c_{'+str(j)+',1}$', color = 'g', linestyle = '--')
#                 plt.axvline(x=0.66666, label = r'$c_{'+str(j)+',2}$', color = 'b', linestyle = '--')
#                 plt.text(0.10, 0.95, r'$\boldsymbol{q}_{'+str(j)+',1}$', fontsize=14, verticalalignment='top')
#                 plt.text(0.50, 0.95, r'$\boldsymbol{q}_{'+str(j)+',2}$', fontsize=14, verticalalignment='top')
#                 plt.text(0.90, 0.95, r'$\boldsymbol{q}_{'+str(j)+',3}$', fontsize=14, verticalalignment='top')
#                 df = pd.DataFrame(np.column_stack((x_quant[:,j],np.argmax(outputs[j][0],axis=1))),columns=['continuous','discrete'])
#                 grouped_data = df.groupby('discrete', as_index=False)['continuous']
#                 results = [grouped_data.max(),grouped_data.min()]
#                 results[0]['continuous'].values[::-1].sort()
#                 results[1]['continuous'].values[::-1].sort()
#                 moyennes = np.column_stack((results[0]['continuous'].values[1:,np.newaxis],results[1]['continuous'].values[:-1,np.newaxis])).mean(axis=1)
#                 for k in reversed(range(moyennes.shape[0])):
#                     plt.axvline(x=moyennes[k], label = r'$\hat{c}_{'+str(j)+','+str(moyennes.shape[0]-k+1)+'}$', color = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'][k], linestyle = '-')
#                 plt.legend(loc='center right')
#                 if (self.epochs == 5):
#                     tikz_save('feature_'+str(j)+'_iteration_'+str(self.epochs)+'.tex', figureheight = '\\figureheight',figurewidth='\\figurewidth')
#                 if (self.epochs == 300):
#                     tikz_save('feature_'+str(j)+'_iteration_'+str(self.epochs)+'.tex', figureheight = '\\figureheight',figurewidth='\\figurewidth')
#                 plt.show()
        
        self.losses.append(evaluate_disc(prob = outputs))
        
        if self.losses[-1] < self.best_criterion:
            self.best_outputs = outputs
            self.best_criterion = self.losses[-1]

# Results

## Discretization of each continuous feature

In [10]:
from itertools import chain
results = []

for rep in range(replicates):
    
    # Simulate data
    x_quant, y = generate_data(seed=rep+40)
    
    # Set up a new model
    
    liste_inputs_quant, liste_layers_quant,liste_layers_quant_inputs = initialize_neural_net()
    output = Dense(1,activation='sigmoid')(concatenate(liste_layers_quant_inputs))
    model = Model(inputs = liste_inputs_quant,outputs = [output])
    #opt = optimizers.Adadelta(lr=6.0, rho=0.98, epsilon=None, decay=0.0)
    #opt = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
    opt = optimizers.RMSprop(lr=0.5, rho=0.9, epsilon=None, decay=0.0)
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    # Train the model
    history = LossHistory()
    callbacks = [ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0),history]
    model.fit(list(x_quant.T),y,epochs=400, batch_size=30, verbose=1, callbacks = callbacks)
    
    # Get the best outputs
    results.append([])
    for j in range(d1):
        df = pd.DataFrame(np.column_stack((x_quant[:,j],np.argmax(history.best_outputs[j][0],axis=1))),columns=['continuous','discrete'])
        grouped_data = df.groupby('discrete', as_index=False)['continuous']
        results[rep].append([grouped_data.max(),grouped_data.min()])

Epoch 1/400
1000/1000 [==============================] - 1s 673us/step - loss: 0.8223 - acc: 0.5120
Epoch 2/400
1000/1000 [==============================] - 0s 38us/step - loss: 0.7593 - acc: 0.5060
Epoch 3/400
1000/1000 [==============================] - 0s 39us/step - loss: 0.7854 - acc: 0.4880
Epoch 4/400
  30/1000 [..............................] - ETA: 0s - loss: 0.6365 - acc: 0.6667

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

1000/1000 [==============================] - 0s 43us/step - loss: 0.7557 - acc: 0.5150
Epoch 5/400
1000/1000 [==============================] - 0s 37us/step - loss: 0.7950 - acc: 0.4820
Epoch 6/400
1000/1000 [==============================] - 0s 37us/step - loss: 0.7473 - acc: 0.5070
Epoch 7/400
  30/1000 [..............................] - ETA: 0s - loss: 1.6051 - acc: 0.2667

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

1000/1000 [==============================] - 0s 44us/step - loss: 0.8102 - acc: 0.4840
Epoch 8/400
1000/1000 [==============================] - 0s 35us/step - loss: 0.7955 - acc: 0.4880
Epoch 9/400
1000/1000 [==============================] - 0s 37us/step - loss: 0.7791 - acc: 0.5060
Epoch 10/400
  30/1000 [..............................] - ETA: 0s - loss: 0.8352 - acc: 0.5000

...

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

Epoch 398/400
1000/1000 [==============================] - 0s 38us/step - loss: 0.5979 - acc: 0.6970
Epoch 399/400
1000/1000 [==============================] - 0s 37us/step - loss: 0.5979 - acc: 0.6970


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

Epoch 400/400
1000/1000 [==============================] - 0s 37us/step - loss: 0.5979 - acc: 0.6970


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [23]:
liste_layers_quant[0].get_weights()

[array([[ 25.014994 , -10.246944 ,  -2.5147095]], dtype=float32),
 array([-5.381966 , 17.020435 , 10.6920805], dtype=float32)]

In [22]:
plt.xlim((0,1))
plt.ylim((0,1))
for k in range(history.best_outputs[j][0].shape[1]):
    plt.plot(np.sort(x_quant[:,j]),history.best_outputs[j][0][np.argsort(x_quant[:,j]),k], color = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'][k+2])
df = pd.DataFrame(np.column_stack((x_quant[:,j],np.argmax(history.best_outputs[j][0],axis=1))),columns=['continuous','discrete'])
plt.show()


FileNotFoundError: ignored

In [0]:
estim_c_2 = []
estim_c_1 = []
for rep in range(10):
    for j in range(d1):
        results[rep][j][0]['continuous'].values[::-1].sort()
        results[rep][j][1]['continuous'].values[::-1].sort()
        moyennes = np.column_stack((results[rep][j][0]['continuous'].values[1:,np.newaxis],results[rep][j][1]['continuous'].values[:-1,np.newaxis])).mean(axis=1)
        if len(moyennes)==2:
            estim_c_2.append(moyennes[0])
            estim_c_1.append(moyennes[1])

In [0]:
estim_c_2

[0.6655325876344982,
 0.666284657006778,
 0.6662464903860197,
 0.6661907598724586,
 0.6662001567595384,
 0.6663616409075533,
 0.6641870243841485,
 0.6657987175891495,
 0.6659752123705766,
 0.6663360980552304,
 0.6659425619287234,
 0.6660440514487367,
 0.6661924271450909,
 0.6659856172791747,
 0.6659759539436546,
 0.6662328969290401,
 0.6653311930296738,
 0.6662246943831316,
 0.6656785475606273,
 0.6659019294727226]

In [0]:
estim_c_2_previous_10000 = [
    0.6672096297443226,
 0.6658690520930759,
 0.6655029565759243,
 0.6665644358390725,
 0.6652714808974778,
 0.6658288599641675,
 0.6664737817512818,
 0.6658455066465907,
 0.6662530567935727,
 0.666240450529812,
 0.6655041697206,
 0.664984698295971,
 0.6647561265491362,
 0.6663295634277331,
 0.6661851291576448,
 0.6679607955548954,
 0.66623257671893,
 0.6662078986263408,
 0.6656608950659391,
 0.6644521835183468,
0.6661305919941549,
 0.666106662164867,
 0.6663510155858305,
 0.6656342448600621,
 0.6655151176839087,
 0.6664444857520264,
 0.6657423070472644,
 0.6659127323065461,
 0.6660471579232436,
 0.6666085677735107,
 0.6647037788811443,
 0.6680375578792432,
 0.6657116622118637,
 0.6659287379210329,
 0.6676008792697218,
 0.6658891368572595,
 0.6659189750365977,
 0.664891355989736,
 0.6637758937854943,
 0.6656396326877034,
0.6657696556491457,
 0.6657704621001952,
 0.6664690889082279,
 0.6657261585068963,
 0.6661440228858182,
 0.6649891573241458,
 0.6656234638538232,
 0.665964193094543,
 0.6660705236735663,
 0.6662951644992559,
 0.6673171352093334,
 0.6661178722552126,
 0.6659848471909509,
 0.6659354183679145,
 0.6667898736481362,
 0.6663455096327259,
 0.665848006291033,
 0.6666308151154239,
 0.6659530103359128,
 0.6661431296386526,
0.6650371156334989,
 0.6648724436673419,
 0.6663251907869568,
 0.6668195850769136,
 0.6660494823219858,
 0.6658568908205188,
 0.6650341976862919,
 0.6658257213705968,
 0.6658240799411275,
 0.6659116824005149,
 0.6658446639778506,
 0.6655991557508187,
 0.6654850849279068,
 0.6660342521573468,
 0.6660598635459393,
 0.6652456332279789,
 0.6647858880983485,
 0.6660476614320381,
 0.6668394595480607,
 0.6660202310682597,
 0.6655325876344982,
 0.666284657006778,
 0.6662464903860197,
 0.6661907598724586,
 0.6662001567595384,
 0.6663616409075533,
 0.6641870243841485,
 0.6657987175891495,
 0.6659752123705766,
 0.6663360980552304,
 0.6659425619287234,
 0.6660440514487367,
 0.6661924271450909,
 0.6659856172791747,
 0.6659759539436546,
 0.6662328969290401,
 0.6653311930296738,
 0.6662246943831316,
 0.6656785475606273,
 0.6659019294727226
]

In [0]:
estim_c_2_previous_1000 = [
    0.6666649361727628, 0.7054363722495351, 0.6414569705220559,
    0.6505031194020103, 0.649473387438559, 0.6683765728288973,
    0.6717092333146805, 0.646731201282551, 0.6671321076912675,
    0.661859603463256, 0.6964471978529386, 0.6615502406578442,
    0.7002466426316314, 0.6547194344207551, 0.675418411219417,
    0.6414455267720114, 0.6588844844156297, 0.6773322551041605,
    0.6652684610632329, 0.6775652030131648, 0.6713561950274713,
    0.6691290080255745, 0.6531535805080115, 0.6491023845353021,
    0.6202667998720162, 0.6615663240776708, 0.6566711535979076,
    0.6447746346311238, 0.6703017273372043, 0.6893221926363137,
    0.6400829199724849, 0.6620845142598565, 0.6361591932396171,
    0.657638916278978, 0.6495705597346737, 0.664520844372062,
    0.6479601385903906, 0.6596400351431799, 0.674864498620214,
    0.6667923497876894, 0.676954976198862, 0.6911919856712487,
    0.6626176551913426, 0.6438272333321158, 0.6632844708623412,
    0.6871721475622357, 0.6467450220585466, 0.6595524808024933,
    0.658256198915519, 0.6586083902362634, 0.7231623247479192,
    0.6632008740412081, 0.6860067654917335, 0.631900338671033,
    0.6650370978456261, 0.6992219396955452, 0.6672345706076804,
    0.7021788074555109, 0.6353640813262815, 0.6701533154649952,
    0.6496221801052868, 0.6150401038273807, 0.6814261631164954,
    0.6464901642864608, 0.6651568998617281, 0.6131109680335374,
    0.6581615641176466, 0.6685867128665569, 0.7013046228423325,
    0.6464789678835933, 0.6505917613061807, 0.6741398541989446,
    0.671228673631999, 0.6827242311203812, 0.6435768249546103,
    0.6908135467225647, 0.6313625465094432, 0.6141874908485452,
    0.6828872191862008, 0.6309031100261521, 0.6702619631873552,
    0.6699320410722545, 0.6568882483057292, 0.6862631414504905,
    0.6057942133706783, 0.6077264580582442, 0.6787210388204246,
    0.6880650726460569, 0.6918282471835009, 0.6891254320918808,
    0.6619411789191432, 0.6912359322924968, 0.6400239163487074,
    0.6636478763774041, 0.6218796502543036, 0.6352293495315644,
    0.6733226413018987, 0.6413574375974043, 0.639520168020177,
    0.6074506392925269, 0.6526613424653886, 0.6687101355204266,
    0.6357735404618048, 0.6586667575669232, 0.639047394876866,
    0.6643920669276806, 0.6883098393907084, 0.6044303551696126,
    0.6482337702323112, 0.6391412658061351, 0.6873388014805515,
    0.6740770904117124, 0.6514218328526467
]

In [0]:
len(estim_c_2_previous_10000)

100

In [0]:
import scipy.stats as st

In [0]:
#st.t.interval(0.95, len(estim_c_1_previous[:100])-1, loc=np.mean(estim_c_1_previous[:100]), scale=st.sem(estim_c_1_previous[:100]))

In [0]:
st.t.interval(0.95, len(estim_c_2_previous_10000[:100])-1, loc=np.mean(estim_c_2_previous_10000[:100]), scale=st.sem(estim_c_2_previous_10000[:100]))

(0.6658064322705357, 0.66607262274526)

In [0]:
# Helper functions for TF Graph visualization
from IPython.display import clear_output, Image, display, HTML
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
graph_def = tf.get_default_graph().as_graph_def()
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)

Output hidden; open in https://colab.research.google.com to view.